In [ ]:
!pip install -q diffusers transformers accelerate torch trimesh pillow flask flask-ngrok
!pip install -q import-ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
print(os.listdir('.'))

['Untitled2.ipynb', 'Lab9.ipynb', '1.ipynb', 'Untitled4.ipynb', 'Untitled5.ipynb', 'internship.ipynb', 'Untitled21.ipynb', 'Untitled22.ipynb', 'Untitled23.ipynb', 'Untitled1.ipynb', 'Untitled0.ipynb', 'shape_e_image_to_3d_colab.ipynb', '.ipynb_checkpoints', 'sshape_e_text_to_3d_colab.ipynb']


In [ ]:
!pip install -q import-ipynb

In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/sshape_e_text_to_3d_colab.ipynb"   :contentReference[oaicite:4]{index=4}
%run "/content/drive/MyDrive/Colab Notebooks/shape_e_image_to_3d_colab.ipynb"


Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 11.12 MiB/s, done.
Resolving deltas: 100% (43/43), done.
Updating files: 100% (224/224), done.
/content/drive/MyDrive/Colab Notebooks/shap-e
Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-xans2euu/clip_dad7bfe46d744630b022a12b82b5cd05
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-xans2euu/clip_dad7bfe46d744630b022a12b82b5cd05
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Running setup.py develop for shap-e


/content/drive/MyDrive/Colab Notebooks/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/drive/MyDrive/Colab Notebooks/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/drive/MyDrive/Colab Notebooks/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/drive/MyDrive/Colab Notebooks/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:19<00:00, 46.8MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
import import_ipynb                                  # installs the loader
import sshape_e_text_to_3d_colab as text_nb           # name matches filename minus .ipynb
import shape_e_image_to_3d_colab as image_nb

In [ ]:
from flask import Flask, request, send_file, render_template_string
from flask_ngrok import run_with_ngrok
import io

app = Flask(__name__)
run_with_ngrok(app)   # Exposes via ngrok at runtime

# Simple upload form
FORM = """
<form method="post" enctype="multipart/form-data">
  <input type="radio" name="mode" value="text" checked>Text<br>
  <textarea name="prompt" placeholder="Enter prompt"></textarea><br>
  <input type="radio" name="mode" value="image">Image<br>
  <input type="file" name="file"><br>
  <button type="submit">Output</button>
</form>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        if request.form["mode"] == "text":
            # import option
            mesh = gen_from_text(request.form["prompt"])
        else:
            # Save the input image for processing
            imgfile = request.files["file"]
            imgfile.save("image-o.png")
            mesh = gen_from_image("image-o.png")

        # Stream the it back to the interface
        buf = io.BytesIO()
        mesh.export(buf, file_type="obj")
        buf.seek(0)
        return send_file(
            buf,
            as_attachment=True,
            download_name="model.obj"
        )  # streams file for download :contentReference[oaicite:1]{index=1}

    return render_template_string(FORM)


In [ ]:
app.py()